# Aggregate the OGGM raw files to regional or global summed up csv files


*Note that the paths of this notebook only work if you are running the notebook from the OGGM cluster*


In [1]:
import xarray as xr
import pandas as pd
import progressbar
import numpy as np
from oggm.utils import mkdir
import matplotlib.pyplot as plt
import json
import os


In [2]:
%pwd

'/home/www/oggm/oggm-standard-projections/oggm-standard-projections-csv-files/notebooks'

In [18]:
dirpath = '/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/'
outputpath = '/home/www/oggm/oggm-standard-projections/oggm-standard-projections-csv-files/1.6.1'
# we only aggregate here `w5e5_gcm_merged`, but `gcm_from_2000` is also available in the raw per-glacier datafiles
historical_future_option = 'w5e5_gcm_merged'

# this is the only preprocessed version (i.e., we match the geodetic observations on any individual glacier)
exp = 'match_geod_pergla'
oggm_version = 'oggm_v1.6.1_2023.3'

rgi_meta = pd.read_hdf('/home/www/oggm/rgi/rgi62_stats.h5')
rgi_meta = rgi_meta.loc[rgi_meta.Connect != 2]

## A. Common running glacier aggregations for all projections, until 2100 and until 2300, i.e. `common_running_2100_2300`
- the same for only until 2100 is done in Section B further below

### A. 1: GCMs - missing glaciers overview  

In [4]:
allfiles = []
for root, dirs, files in os.walk(dirpath):
    for file in files:
        if (file.endswith(".nc")) and ('run_hydro' in file) and (historical_future_option in file) and ('_0_1000' in file) and ('basin' not in file):
             allfiles.append(os.path.join(root, file))

df_meta = pd.DataFrame()
invalid_per_reg = {}  
meta_per_reg = {}

In [6]:
run = True
if run:
    # this takes very long !!!
    invalid_per_reg = {}  
    meta_per_reg = {}
    for f in allfiles: #progressbar.progressbar(allfiles):
        ename = f.replace(dirpath, '')
        ss = ename.split('/')
        cmip = ss[0]
        endyr = ss[1]
        rgi_reg = ss[2]
        gcm = ss[3].split('_')[5].upper() #ss[2]
        if 'ISIMIP3b' in cmip:
            ssp = ss[3].split('_')[7]
            bc = 'ISIMIP3b:1979-2014'
        else:
            ssp = ss[3].split('_')[6]
            bc = '2000-2019'

        end0,end1 = ss[3].split('_')[-2:]
        end = end0+'_'+end1
        ff = f[:-len(end)]+'*'

        #if ssp not in ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']:
        #    continue

        # exp is the same everywhere, so we don't need as an id... 
        run_id = f'{cmip}:{endyr}:{gcm}:{ssp}:{rgi_reg}'
        df_meta.loc[run_id, 'cmip'] = cmip
        df_meta.loc[run_id, 'gcm'] = gcm
        df_meta.loc[run_id, 'scenario'] = ssp
        df_meta.loc[run_id, 'rgi_reg'] = rgi_reg
        df_meta.loc[run_id, 'end_year'] = 0
        df_meta.loc[run_id, 'bias_correction'] = bc

        df_meta.loc[run_id, 'perc_area_missing'] = 0

        df_meta.loc[run_id, 'fpath'] = ff

        if rgi_reg not in invalid_per_reg:
            invalid_per_reg[rgi_reg] = set()

        if rgi_reg not in meta_per_reg:
            meta_per_reg[rgi_reg] = rgi_meta.loc[rgi_meta['O1Region'] == rgi_reg[-2:]]

        with xr.open_mfdataset(ff) as ds:
            ds_t = ds.isel(time=-1).volume.load()
        df_meta.loc[run_id, 'end_year'] = str(int(ds_t.time))
        missing_ids = ds_t.rgi_id[ds_t.isnull()].data
        perc = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum() / meta_per_reg[rgi_reg]['Area'].sum() 
        df_meta.loc[run_id, 'perc_area_missing'] = perc
        invalid_per_reg[rgi_reg] = invalid_per_reg[rgi_reg].union(missing_ids)
        if f == allfiles[1000] or f==allfiles[2000]:
            print(f)
    df_meta['exp'] = exp
    df_meta['historical_future_option'] = historical_future_option
    df_meta['oggm_version'] = oggm_version
    df_meta.to_csv(f'{outputpath}/common_running_2100_2300/metadata.csv')
else:
    df_meta = pd.read_csv(f'{outputpath}/common_running_2100_2300/metadata.csv')

/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/CMIP6/2100/RGI12/run_hydro_w5e5_gcm_merged_CAMS-CSM1-0_ssp585_bc_2000_2019_Batch_0_1000.nc
/home/www/oggm/oggm-standard-projections/oggm_v16/2023.3/CMIP5/2100/RGI06/run_hydro_w5e5_gcm_merged_GISS-E2-R_rcp26_bc_2000_2019_Batch_0_1000.nc


In [20]:
dds = df_meta.sort_values('perc_area_missing', ascending=False)
dds[dds['perc_area_missing'] > 0.05].rgi_reg.unique()

array(['RGI12', 'RGI06', 'RGI08'], dtype=object)

In [21]:
dds

,cmip,gcm,scenario,rgi_reg,end_year,bias_correction,perc_area_missing,fpath,exp,historical_future_option,oggm_version
CMIP5:2100:CANESM2:rcp26:RGI12,CMIP5,CANESM2,rcp26,RGI12,2100,2000-2019,0.121243,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP6:2100:NORESM2-MM:ssp126:RGI12,CMIP6,NORESM2-MM,ssp126,RGI12,2100,2000-2019,0.120892,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP5:2100:IPSL-CM5A-LR:rcp26:RGI12,CMIP5,IPSL-CM5A-LR,rcp26,RGI12,2100,2000-2019,0.119703,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP5:2100:MPI-ESM-LR:rcp26:RGI12,CMIP5,MPI-ESM-LR,rcp26,RGI12,2100,2000-2019,0.119397,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP5:2100:CANESM2:rcp45:RGI12,CMIP5,CANESM2,rcp45,RGI12,2100,2000-2019,0.119350,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
...,...,...,...,...,...,...,...,...,...,...,...
CMIP6:2300:ACCESS-ESM1-5:ssp585:RGI07,CMIP6,ACCESS-ESM1-5,ssp585,RGI07,2300,2000-2019,0.000000,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP6:2300:IPSL-CM6A-LR:ssp126:RGI07,CMIP6,IPSL-CM6A-LR,ssp126,RGI07,2300,2000-2019,0.000000,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP6:2300:CANESM5:ssp126:RGI07,CMIP6,CANESM5,ssp126,RGI07,2300,2000-2019,0.000000,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3
CMIP6:2300:IPSL-CM6A-LR:ssp534-over:RGI07,CMIP6,IPSL-CM6A-LR,ssp534-over,RGI07,2300,2000-2019,0.000000,/home/www/oggm/oggm-standard-projections/oggm_...,match_geod_pergla,w5e5_gcm_merged,oggm_v1.6.1_2023.3


In [22]:
for k, v in invalid_per_reg.items():
    invalid_per_reg[k] = list(v)
with open(f'{outputpath}/common_running_2100_2300/rgi_ids_missing.json', 'w') as f:
    json.dump(invalid_per_reg, f)

In [23]:
# these are now the common running glaciers over all scenarios and GCMs and all cmip variants going until 2100 or until 2300
odf = pd.DataFrame()
for rgi_reg, missing_ids in invalid_per_reg.items():
    odf.loc[rgi_reg, 'n_glaciers'] = len(meta_per_reg[rgi_reg])
    odf.loc[rgi_reg, 'n_missing_glaciers'] = len(missing_ids)
    odf.loc[rgi_reg, 'rgi_area_km2'] = meta_per_reg[rgi_reg]['Area'].sum() 
    odf.loc[rgi_reg, 'missing_area_km2'] = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum()
    odf.loc[rgi_reg, 'missing_area_perc'] = odf.loc[rgi_reg, 'missing_area_km2'] / odf.loc[rgi_reg, 'rgi_area_km2']
odf[['n_glaciers', 'n_missing_glaciers']] = odf[['n_glaciers', 'n_missing_glaciers']].astype(int)
odf = odf.sort_values(by='missing_area_perc', ascending=False)
odf

,n_glaciers,n_missing_glaciers,rgi_area_km2,missing_area_km2,missing_area_perc
RGI12,1888,409,1306.992,160.691,0.122947
RGI06,568,420,11059.700,963.393,0.087108
RGI08,3417,1152,2949.103,237.021,0.080371
RGI10,5151,243,2410.051,94.172,0.039075
RGI05,19306,1662,89717.066,1467.785,0.016360
RGI07,1615,48,33958.934,238.343,0.007019
RGI13,54429,862,49303.415,311.203,0.006312
RGI14,27988,501,33568.298,102.782,0.003062
RGI19,2752,633,132867.219,377.671,0.002842
RGI04,7415,313,40888.228,100.893,0.002468


In [24]:
odf.to_csv(f'{outputpath}/common_running_2100_2300/missing_region_overview.csv')
odf.to_html(f'{outputpath}/common_running_2100_2300/missing_region_overview.html')

### A. 2: GCMs - aggregation

In [25]:
base_dir =f'{outputpath}/common_running_2100_2300'
df_meta = pd.read_csv(f'{base_dir}/metadata.csv', index_col = 'Unnamed: 0')
import json
with open(f'{base_dir}/rgi_ids_missing.json', 'r') as f:
    invalid_per_reg = json.load(f)

In [14]:
for cmip in df_meta.cmip.unique():
    #print(cmip, flush=True)
    if cmip == 'CMIP6':
        end_years = [2100, 2300]
    else:
        end_years = [2100]
    for end_year in end_years:
        for rgi_reg in sorted(df_meta.rgi_reg.unique()): #progressbar.progressbar(
            for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
                df_meta_s = df_meta.loc[(df_meta.end_year == end_year) & (df_meta.cmip == cmip) & (df_meta.rgi_reg == rgi_reg) & (df_meta.scenario == scenario)]
                odf_v = pd.DataFrame()
                odf_a = pd.DataFrame()
                ods = []
                gcms = []
                for i, s in df_meta_s.iterrows():
                    with xr.open_mfdataset(s.fpath) as ds:
                        ds = ds[['volume', 'area']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        #ds = ds[['volume', 'area', 'length']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        odf_v[s.gcm] = ds.volume.sum(dim='rgi_id').to_series()
                        odf_a[s.gcm] = ds.area.sum(dim='rgi_id').to_series()
                        ods.append(ds)
                        gcms.append(s.gcm)
                ods = xr.concat(ods, 'gcm')

                # we don't like to have these kind of summary statistics as the user does not know
                # whether the different projections are gaussian distributed ... 
                #ods.mean(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_avg_{scenario}.nc')
                #ods.std(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_std_{scenario}.nc')
                odir =  f'{base_dir}/volume/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_v.to_csv(odir + f'{scenario}.csv')
                odir = f'{base_dir}/area/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_a.to_csv(odir + f'{scenario}.csv')

In [31]:
df_meta.end_year.unique()

array([2100, 2300])

In [34]:
# globally: sum over all RGI regions
for var in ['volume', 'area']:
    for cmip in df_meta.cmip.unique():
        if cmip == 'CMIP6':
            end_years = [2100, 2300]
        else:
            end_years = [2100]
        for end_year in end_years:
            odir = f'{base_dir}/{var}/{cmip}/{end_year}/global/'
            mkdir(odir)
            for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
                odf = 0
                for rgi_reg in sorted(df_meta.rgi_reg.unique()):
                    idir = f'{base_dir}/{var}/{cmip}/{end_year}/{rgi_reg}/'
                    df = pd.read_csv(idir + f'/{scenario}.csv', index_col=0)
                    odf += df
                odf.to_csv(odir + f'/{scenario}.csv')

## B. Common running glacier aggregations for all projections until 2100, i.e. `common_running_2100`

### B. 1: GCMs - missing glaciers overview  

In [ ]:
allfiles = []
for root, dirs, files in os.walk(dirpath):
    if not '/2300/' in root:
        for file in files:
            if (file.endswith(".nc")) and ('run_hydro' in file) and (historical_future_option in file) and ('_0_1000' in file) and ('basin' not in file):
                 allfiles.append(os.path.join(root, file))

df_meta = pd.DataFrame()
invalid_per_reg = {}  
meta_per_reg = {}

In [ ]:
# this takes very long !!!
invalid_per_reg = {}  
meta_per_reg = {}
for f in allfiles: #progressbar.progressbar(allfiles):
    ename = f.replace(dirpath, '')
    ss = ename.split('/')
    cmip = ss[0]
    endyr = ss[1]
    rgi_reg = ss[2]
    gcm = ss[3].split('_')[5].upper() #ss[2]
    if 'ISIMIP3b' in cmip:
        ssp = ss[3].split('_')[7]
        bc = 'ISIMIP3b:1979-2014'
    else:
        ssp = ss[3].split('_')[6]
        bc = '2000-2019'
    
    end0,end1 = ss[3].split('_')[-2:]
    end = end0+'_'+end1
    ff = f[:-len(end)]+'*'
    
    #if ssp not in ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']:
    #    continue
    
    # exp is the same everywhere, so we don't need as an id... 
    run_id = f'{cmip}:{endyr}:{gcm}:{ssp}:{rgi_reg}'
    df_meta.loc[run_id, 'cmip'] = cmip
    df_meta.loc[run_id, 'gcm'] = gcm
    df_meta.loc[run_id, 'scenario'] = ssp
    df_meta.loc[run_id, 'rgi_reg'] = rgi_reg
    df_meta.loc[run_id, 'end_year'] = 0
    df_meta.loc[run_id, 'bias_correction'] = bc

    df_meta.loc[run_id, 'perc_area_missing'] = 0
    
    df_meta.loc[run_id, 'fpath'] = ff
    
    if rgi_reg not in invalid_per_reg:
        invalid_per_reg[rgi_reg] = set()
    
    if rgi_reg not in meta_per_reg:
        meta_per_reg[rgi_reg] = rgi_meta.loc[rgi_meta['O1Region'] == rgi_reg[-2:]]
    
    with xr.open_mfdataset(ff) as ds:
        ds_t = ds.isel(time=-1).volume.load()
    df_meta.loc[run_id, 'end_year'] = str(int(ds_t.time))
    missing_ids = ds_t.rgi_id[ds_t.isnull()].data
    perc = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum() / meta_per_reg[rgi_reg]['Area'].sum() 
    df_meta.loc[run_id, 'perc_area_missing'] = perc
    invalid_per_reg[rgi_reg] = invalid_per_reg[rgi_reg].union(missing_ids)
    if f == allfiles[100]:
        print(f)
df_meta['exp'] = exp
df_meta['historical_future_option'] = historical_future_option
df_meta['oggm_version'] = oggm_version
df_meta.to_csv(f'{outputpath}/common_running_2100/metadata.csv')

In [ ]:
dds = df_meta.sort_values('perc_area_missing', ascending=False)
dds[dds['perc_area_missing'] > 0.05].rgi_reg.unique()

In [ ]:
dds

In [ ]:
for k, v in invalid_per_reg.items():
    invalid_per_reg[k] = list(v)
with open(f'{outputpath}/common_running_2100/rgi_ids_missing.json', 'w') as f:
    json.dump(invalid_per_reg, f)

# these are now the common running glaciers over all scenarios and GCMs and all cmip variants going until 2100 or until 2300
odf = pd.DataFrame()
for rgi_reg, missing_ids in invalid_per_reg.items():
    odf.loc[rgi_reg, 'n_glaciers'] = len(meta_per_reg[rgi_reg])
    odf.loc[rgi_reg, 'n_missing_glaciers'] = len(missing_ids)
    odf.loc[rgi_reg, 'rgi_area_km2'] = meta_per_reg[rgi_reg]['Area'].sum() 
    odf.loc[rgi_reg, 'missing_area_km2'] = meta_per_reg[rgi_reg].loc[missing_ids]['Area'].sum()
    odf.loc[rgi_reg, 'missing_area_perc'] = odf.loc[rgi_reg, 'missing_area_km2'] / odf.loc[rgi_reg, 'rgi_area_km2']
odf[['n_glaciers', 'n_missing_glaciers']] = odf[['n_glaciers', 'n_missing_glaciers']].astype(int)
odf = odf.sort_values(by='missing_area_perc', ascending=False)
odf

In [ ]:
odf.to_csv(f'{outputpath}/common_running_2100/missing_region_overview.csv')
odf.to_html(f'{outputpath}/common_running_2100/missing_region_overview.html')

### B. 2: GCMs - aggregation

In [32]:
base_dir =f'{outputpath}/common_running_2100'
df_meta = pd.read_csv(f'{base_dir}/metadata.csv', index_col = 'Unnamed: 0')
with open(f'{base_dir}/rgi_ids_missing.json', 'r') as f:
    invalid_per_reg = json.load(f)

In [ ]:
for cmip in df_meta.cmip.unique():
    print(cmip, flush=True)
    if cmip == 'CMIP6':
        end_years = [2100]
    else:
        end_years = [2100]
    for end_year in end_years:
        for rgi_reg in sorted(df_meta.rgi_reg.unique()): #progressbar.progressbar(
            for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
                df_meta_s = df_meta.loc[(df_meta.end_year == end_year) & (df_meta.cmip == cmip) & (df_meta.rgi_reg == rgi_reg) & (df_meta.scenario == scenario)]
                odf_v = pd.DataFrame()
                odf_a = pd.DataFrame()
                ods = []
                gcms = []
                for i, s in df_meta_s.iterrows():
                    with xr.open_mfdataset(s.fpath) as ds:
                        ds = ds[['volume', 'area']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        #ds = ds[['volume', 'area', 'length']].load().isel(rgi_id=~ds.rgi_id.isin(invalid_per_reg[s.rgi_reg]))
                        odf_v[s.gcm] = ds.volume.sum(dim='rgi_id').to_series()
                        odf_a[s.gcm] = ds.area.sum(dim='rgi_id').to_series()
                        ods.append(ds)
                        gcms.append(s.gcm)
                ods = xr.concat(ods, 'gcm')

                # I don't like to have these kind of summary statistics as the user does not know whether the different projections are gaussian distributed .. 
                #ods.mean(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_avg_{scenario}.nc')
                #ods.std(dim='gcm').to_netcdf(f'output_1.6.1_2023.3/{exp}/{rgi_reg}/all_gcm_std_{scenario}.nc')
                odir =  f'{base_dir}/volume/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_v.to_csv(odir + f'{scenario}.csv')
                odir = f'{base_dir}/area/{cmip}/{end_year}/{rgi_reg}/'
                mkdir(odir)
                odf_a.to_csv(odir + f'{scenario}.csv')

ISIMIP3b_CMIP6


In [ ]:
# globally: sum over all RGI regions
for var in ['volume', 'area']:
    for cmip in df_meta.cmip.unique():
        odir = f'{base_dir}/{var}/{cmip}/{end_year}/global/'
        mkdir(odir)
        for scenario in sorted(df_meta.loc[(df_meta.cmip==cmip) & (df_meta.end_year == end_year)].scenario.unique()):
            odf = 0
            for rgi_reg in sorted(df_meta.rgi_reg.unique()):
                idir = f'{base_dir}/{var}/{cmip}/{end_year}/{rgi_reg}/'
                df = pd.read_csv(idir + f'/{scenario}.csv', index_col=0)
                odf += df
            odf.to_csv(odir + f'/{scenario}.csv')